## Lab01: Collect and preprocess data

- Name: Nguyen Phat Dat
- Student code: 21127240

***
## How to do and submit your assignment

**Work on your assignment**

You will do your assignment directly in this notebook. First, fill in your name and ID at the beginning of the file. In the notebook, fill in places that say:
```python
#TODO
```

During your work, you can print out the result, create more cells to test, or create more functions to handle things. Please note that <font color=red>you are not allowed to delete or modify my code cells</font> (except in the case that mentioned above). Let remove `raise NotImplementedError("not implement")` when running code.

Always press `Ctrl + S` in order to save your work.

**Notes:**

    *  Copy means zero
    *  You have to submit your work on time. No exception
    *  Any questions about your grade are based on the assignment submitted on Moodle
    *  Wrong submission takes you -2 points

**Submission guideline**

When grading your assignment, I will choose `Kernel` - `Restart Kernel & Run All Cells` in order to restart the kernel and run all cells in your notebook. Therefore, you should do that before submitting to ensure that the outputs are all as expected.

After that, you make a submited direction as follow:

- Folder `StudentCode` (e.g. If your student code is 1234567, then your folder is `1234567`)
    - File `<StudentCode>.ipynb` (e.g. If your student code is 1234567, then your file is `1234567.ipynb`)

Finally, you compress your folder (`StudentCode`) and submit on Moodle. **The extension of the file is nothing else but `.zip`.**

<font color=red>Please strictly follow the submission rules.</font>

# 1. Set-up environment

In [1]:
#Necessary Packages
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
# YOUR CODE HERE (OPTION)
# If you need other support packages

# lxml parser (bs4), if not installed: 'pip install lxml'
from threading import Thread # use: parallel requesting

# 2. Collect data from a website by parsing HTML (3p)

In this section, you are going to collect data from a website simulating the sale of Pokemon. I have prepared all the needed links in a file (`pokemon.txt`), and you have to crawl data from these links. The expected output is a `dataframe` with the following fields:

+ `SKU`: ID of Pokemon
+ `Name`: Name of Pokemon
+ `Price`: Price of Pokemon
+ `InStock`: Quantity of Pokemons in stock
+ `Categories`: The category of Pokemon
+ `Tags`: Tags of Pokemon

Your mission is to complete function `collect_data` with `course_urls_file` as your input parameter. The output should look like `pokemon_example.csv` (I list out some examples so you can easily imagine your work).

In [2]:
pokemon_example = pd.read_csv('pokemon_example.csv')
pokemon_example

,SKU,Name,Price,InStock,Categories,Tags
0,4391,Bulbasaur,63.0,45,"Pokemon, Seed","bulbasaur, Overgrow, Seed"
1,7227,Ivysaur,87.0,142,"Pokemon, Seed","ivysaur, Overgrow, Seed"
2,7036,Venusaur,105.0,30,"Pokemon, Seed","Overgrow, Seed, venusaur"
3,9086,Charmander,48.0,206,"Lizard, Pokemon","Blaze, charmander, Lizard"
4,6565,Charmeleon,165.0,284,"Flame, Pokemon","Blaze, charmeleon, Flame"


In [3]:
def parallel_requesting(url_list, returnDF, index):
    '''
    Perform parallel requesting, with each thread runs on the separate url_list.
    
    returnDF is the list of n (number of thread) DataFrame representing data from each portion of the url
    and acts as the return value from thread 'index'.
    '''

    # the parameter returnDF is the shared list of every thread with length equals to number of threads
    # this help each thread modifies distinct item in the list to avoid conflict (race condition, critical section...)

    #init empty list to store the values of each attribute.
    SKU = []
    Names = []
    Prices = []
    InStocks = []
    Categories = []
    Tags = []
    
    for url in url_list:
        #TODO
        soup = None
        summary = None

        success = False
        for re in range(5): # if request fail 5 times, stop requesting
            respone = requests.get(url)
            if respone: 
                # if the request succeeded
                # start parsing with beautiful soup 
                soup = BeautifulSoup(respone.content, 'html.parser') 
                results = soup.find(id="content")
                if not results:
                    continue
                summary = results.find("div", class_="summary entry-summary")
                if not summary:
                    continue
                success = True
                break
            else:
                # if the request failed
                # wait for 1 sec until next request
                time.sleep(1) 
        if not success:
            # another way: raise exception. But it will end the program
            print(f"Request {url} failed multiple times\nPlease check the internet connection and try again!")
            break
        # find information of the pokemon

        # name
        pokeName = summary.find('h1', class_='product_title entry-title').text
        # price
        pokePrice = summary.find('p', class_='price').text[1:] # extract the price symbol
        # stock number
        pokeStock = summary.find('p', class_='stock in-stock').text.removesuffix(' in stock')
        # SKU
        pokeSKU = summary.find('span', class_='sku').text
        # categories
        pokeCategories = summary.find('span', class_='posted_in').text.removeprefix('Categories: ')
        # tags
        pokeTags = summary.find('span', class_='tagged_as').text.removeprefix('Tags: ')

        # append
        SKU.append(pokeSKU)
        Names.append(pokeName)
        Prices.append(pokePrice)
        InStocks.append(pokeStock)
        Categories.append(pokeCategories)
        Tags.append(pokeTags)

    # returnDF[index] acts as the return value after thread 'index' completed
    returnDF[index] = pd.DataFrame({"SKU": SKU,
                         "Name": Names,
                         "Price": Prices,
                         "InStock": InStocks,
                         "Categories": Categories,
                         "Tags": Tags})

In [4]:
def splitURL_N_process(urls, n_thread):
    '''
    Split the url list to n sub list for each of thread run on it.
    And finally merge all results of every thread to get the whole data
    '''
    data = pd.DataFrame()

    NUM_THREAD = int(n_thread) # number of threads: n
    if n_thread <= 0:
        print("The number of threads is not valid!")
        return
    # split the whole url_list into n smaller url_sublist
    url_sublist = []
    st = 0
    en = len(urls) // NUM_THREAD
    for i in range(NUM_THREAD):
        if i != NUM_THREAD - 1:
            url_sublist.append(urls[st:en])
            st = en
            en += len(urls) // NUM_THREAD
        else:
            url_sublist.append(urls[st:])
            break

    results_thr = [None for _ in range(NUM_THREAD)] # contains results of n threads
    thr_list = [None for _ in range(NUM_THREAD)] # contains n Thread objects
    for i in range(NUM_THREAD):
        thr_list[i] = Thread(target=parallel_requesting, args=(url_sublist[i], results_thr, i))
        thr_list[i].start()
    
    # wait for each thread to finish and be terminated
    for j in range(len(thr_list)):
        thr_list[j].join()

    # merge data from all results
    for j in range(NUM_THREAD):
        data = pd.concat([data, results_thr[j]], axis=0)
        
    return data

In [5]:
def collect_data(course_urls_file):
    #load paths from file
    url_file = open(course_urls_file)
    urls = url_file.readlines()
    urls_filtered = [item[:-1] for item in urls]
    
    #init empty list to store the values of each attribute.
    SKU = []
    Names = []
    Prices = []
    InStocks = []
    Categories = []
    Tags = []

    for url in urls_filtered:
        #TODO
        # single thead part: 
        
        soup = None
        summary = None
        success = False
        for re in range(5): # if request fail 5 times, stop requesting
            respone = requests.get(url)
            if respone: 
                # if the request succeeded
                # start parsing with beautiful soup 
                soup = BeautifulSoup(respone.content, 'html.parser') 
                results = soup.find(id="content")
                if not results:
                    continue
                summary = results.find("div", class_="summary entry-summary")
                if not summary:
                    continue
                success = True
                break
            else:
                # if the request failed
                # wait for 1 sec until next request
                time.sleep(1) 
        if not success:
            # another way: raise exception. But it will end the program
            print(f"Request {url} failed multiple times\nPlease check the internet connection and try again!")
            break

        # get information of the pokemon
        # name
        pokeName = summary.find('h1', class_='product_title entry-title').text
        # price
        pokePrice = summary.find('p', class_='price').text[1:] # extract the price symbol
        # stock number
        pokeStock = summary.find('p', class_='stock in-stock').text.removesuffix(' in stock')
        # SKU
        pokeSKU = summary.find('span', class_='sku').text
        # categories
        pokeCategories = summary.find('span', class_='posted_in').text.removeprefix('Categories: ')
        # tags
        pokeTags = summary.find('span', class_='tagged_as').text.removeprefix('Tags: ')
        # append
        SKU.append(pokeSKU)
        Names.append(pokeName)
        Prices.append(pokePrice)
        InStocks.append(pokeStock)
        Categories.append(pokeCategories)
        Tags.append(pokeTags)

        # end part

        # Note: I noticed that each request takes about 1-2s (can be longer if request unsuccessfully)
        # so with 755 urls in the 'pokeon.txt' file, the whole operation could take about 12 to 20 min

        # => solution: instead of request one url each time, we can request multiple urls at the same time 
        # by using multiple threads to do that. Then merge the results to get the whole data
        # However, I'm not sure if I'm allowed to break this for-loop or not, so I'll leave this out 

        # If you want to use it, just comment the above part and uncomment the below part

        # multithread part:

        # tmp_data = splitURL_N_process(urls_filtered, n_thread=20) # take about 1 to 1.5 min
        # SKU = tmp_data['SKU']
        # Names = tmp_data['Name']
        # Prices = tmp_data['Price']
        # InStocks = tmp_data['InStock']
        # Categories = tmp_data['Categories']
        # Tags = tmp_data['Tags']
        # break 

        # end part
    
    data = pd.DataFrame({"SKU": SKU,
                         "Name": Names,
                         "Price": Prices, 
                         "InStock": InStocks,
                         "Categories": Categories,
                         "Tags": Tags})
    
    return data

In [6]:
#TEST
data_pokemon = collect_data("pokemon.txt")
assert data_pokemon.shape == (755, 6)

In [7]:
#Save to csv file with name pokemon.csv
#TODO
data_pokemon.to_csv('pokemon.csv', index=False)
data_pokemon

,SKU,Name,Price,InStock,Categories,Tags
0,4391,Bulbasaur,63.00,45,"Pokemon, Seed","bulbasaur, Overgrow, Seed"
1,7227,Ivysaur,87.00,142,"Pokemon, Seed","ivysaur, Overgrow, Seed"
2,7036,Venusaur,105.00,30,"Pokemon, Seed","Overgrow, Seed, venusaur"
3,9086,Charmander,48.00,206,"Lizard, Pokemon","Blaze, charmander, Lizard"
4,6565,Charmeleon,165.00,284,"Flame, Pokemon","Blaze, charmeleon, Flame"
...,...,...,...,...,...,...
750,1160,Marshadow,107.00,250,"Gloomdweller, Pokemon","Gloomdweller, marshadow, Technician"
751,1771,Poipole,130.00,21,"Poison Pin, Pokemon","Beast Boost, poipole, Poison Pin"
752,9798,Naganadel,30.00,255,"Poison Pin, Pokemon","Beast Boost, naganadel, Poison Pin"
753,1947,Stakataka,190.00,210,"Pokemon, Rampart","Beast Boost, Rampart, stakataka"


In this section, your work is to practice to crawl data using Web API (http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL). This is the data of World Bank which includes demographic data and other statistics related to Population, Employment, Health, GDP, Energy Consumption,... for all countries in the world from 1960 to 2022.

From the following selected indicators:
- `SP.POP.TOTL` - Total population
- `SP.POP.TOTL.FE.IN` - Total female population
- `SP.POP.TOTL.MA.IN` - Total male population
- `SP.DYN.CBRT.IN` - Birth rate
- `SP.DYN.CDRT.IN` - Death rate
- `SP.DYN.LE00.MA.IN` - Average life expectancy of male
- `SP.DYN.LE00.FE.IN` - Average life expectancy of female
- `SE.PRM.ENRR` - Primary school enrollment rate
- `SE.TER.ENRR` - High school enrollment rate
- `SE.PRM.CMPT.ZS` - Primary completion rate
- `SE.ADT.1524.LT.ZS` - Literacy rate of people ages 15-24

You are required to collect data from 7 countries and save to dataframe `data_countries`:
- `US` - United States of America
- `IN` - India
- `CN` - China
- `JP` - Japan
- `CA` - Canada
- `GB` - Great Britain
- `ZA` - South Africa

You can expand your work on collecting data (such as collecting data from other countries and other indicators) by reading: https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-api-documentation

**Hints**:

- Use the based URL: http://api.worldbank.org/v2/
- In order to collect data for each indicator of each country, you can use the URL: "http://api.worldbank.org/v2/countries/{country_code}/indicators/{indicator_code}"
    + `country_code` and `indicator_code` are provided above.
    + For example, you can use the following URL to get the `Total population` of Japan: http://api.worldbank.org/v2/countries/jp/indicators/SP.POP.TOTL

# 3. Collect data using Web API (4p)

In [8]:
data_countries_examples = pd.read_csv("countries_example.csv")
data_countries_examples

,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Male life expectancy,Female life expectancy,"School enrollment, primary","School enrollment, tertiary",Primary completion rate,Literacy rate,Year,Country
0,333287557.0,168266219.0,165021339.0,None,None,None,None,None,None,None,None,2022,USA
1,332031554.0,167550001.0,164481553.0,11.0,10.4,73.5,79.3,None,None,None,None,2021,USA
2,331511512.0,167203010.0,164308503.0,10.9,10.3,74.2,79.9,100.305793762207,87.5676574707031,100.923667907715,None,2020,USA
3,328329953.0,165599805.0,162730147.0,11.4,8.7,76.3,81.4,100.981300354004,87.8887100219727,100.489051818848,None,2019,USA
4,326838199.0,164926348.0,161911851.0,11.6,8.678,76.2,81.2,101.256561279297,88.2991790771484,100.092697143555,None,2018,USA
5,325122128.0,164151818.0,160970309.0,11.8,8.638,76.1,81.1,101.821441650391,88.1673889160156,98.8321990966797,None,2017,USA
6,323071755.0,163224028.0,159847727.0,12.2,8.493,76.1,81.1,101.362861633301,88.8350524902344,None,None,2016,USA
7,320738994.0,162158414.0,158580581.0,12.4,8.44,76.3,81.2,100.299911499023,88.8894119262695,None,None,2015,USA
8,318386329.0,161084758.0,157301571.0,12.5,8.237,76.5,81.3,99.6733779907227,88.6268692016602,None,None,2014,USA
9,316059947.0,160034189.0,156025758.0,12.4,8.215,76.4,81.2,99.455436706543,88.7264175415039,None,None,2013,USA


In [9]:
BASE_URL = 'http://api.worldbank.org/v2/'
COUNTRIES = ["US", "IN", "CN", "JP", "CA", "GB", "ZA"]
INDICATORS = ['SP.POP.TOTL',
             'SP.POP.TOTL.FE.IN',
             'SP.POP.TOTL.MA.IN',
             'SP.DYN.CBRT.IN',
             'SP.DYN.CDRT.IN',
             'SP.DYN.LE00.MA.IN',
             'SP.DYN.LE00.FE.IN',
             'SE.PRM.ENRR',
             'SE.TER.ENRR',
             'SE.PRM.CMPT.ZS',
             'SE.ADT.1524.LT.ZS']

#TODO (option)
# If you need other initializations

In [10]:
def collect_data(countryCode, per_page, start_year, end_year):
    #TODO
    
    temp_ind = 'tempIndicator' # this acts as the placeholder for other indicators

    # the url 
    tmp_url = f'{BASE_URL}countries/{countryCode}/indicators/{temp_ind}'
    # add some arguments
    tmp_url += f'?date={start_year}:{end_year}&per_page={per_page}'

    yearRange = ['None' for _ in range(end_year - start_year + 1)]

    dataList = [yearRange.copy() for _ in range(len(INDICATORS) + 2)]

    for i, ind in enumerate(INDICATORS):
        soup = None
        # just need to replace old indicator with new one instead of recreate the new string
        tmp_url = tmp_url.replace(temp_ind, ind) # replace with current
        temp_ind = ind # old indicator

        success = False
        for re in range(5): # if request fail 5 times, stop requesting
            respone = requests.get(tmp_url)
            if respone:
                # if the request succeeded
                # start parsing with beautiful soup 
                soup = BeautifulSoup(respone.content, 'lxml-xml')
                success = True
                break
            else:
                # if the request failed
                # wait for 1 sec until next request
                time.sleep(1)
        if not success:
            # another way: raise exception. But it will end the program
            print(f"Request {tmp_url} failed multiple times\nPlease check the internet connection and try again!")
            break

        itemList = soup.find_all('data')[1:] # remove the root

        for item in itemList:
            date = int(item.find('date').text)
            value = item.find('value').text
            countryName = item.find('countryiso3code').text

            pos = end_year - date
            dataList[i][pos] = float(value) if value != '' else 'None' # in case there's empty value
            dataList[-2][pos] = date
            dataList[-1][pos] = countryName
            
    return pd.DataFrame({"Total Population":dataList[0],
                 "Female Population":dataList[1],
                 "Male Population":dataList[2],
                 "Birth Rate":dataList[3],
                 "Death Rate":dataList[4],
                 "Male life expectancy":dataList[5],
                 "Female life expectancy ":dataList[6],
                 "School enrollment, primary":dataList[7],
                 "School enrollment, tertiary":dataList[8],
                 "Primary completion rate":dataList[9],
                 "Literacy rate":dataList[10],
                 "Year":dataList[11],
                 "Country":dataList[12]})

In [11]:
def Generate_Countries_Dataset(countryCode_list):
    data = pd.DataFrame()
    for countryCode in countryCode_list:
        data = pd.concat([data, collect_data(countryCode = countryCode, per_page = 100, start_year = 2000, end_year = 2022)], axis=0)
    return data

In [12]:
#TEST
data_countries = Generate_Countries_Dataset(COUNTRIES)
assert data_countries.shape == (161, 13)

In [13]:
# Save to csv file with name coutries.csv
#TODO
data_countries.to_csv('coutries.csv', index=False) # or countries.csv
data_countries

,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Male life expectancy,Female life expectancy,"School enrollment, primary","School enrollment, tertiary",Primary completion rate,Literacy rate,Year,Country
0,333287557.0,168266219.0,165021339.0,None,None,None,None,None,None,None,None,2022,USA
1,332031554.0,167550001.0,164481553.0,11.0,10.4,73.5,79.3,None,None,None,None,2021,USA
2,331511512.0,167203010.0,164308503.0,10.9,10.3,74.2,79.9,100.305794,87.567657,100.923668,None,2020,USA
3,328329953.0,165599805.0,162730147.0,11.4,8.7,76.3,81.4,100.9813,87.88871,100.489052,None,2019,USA
4,326838199.0,164926348.0,161911851.0,11.6,8.678,76.2,81.2,101.256561,88.299179,100.092697,None,2018,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,48556071.0,25379489.0,23176582.0,21.261,12.947,51.891,55.721,106.446632,None,95.654922,None,2004,ZAF
19,48104048.0,25178665.0,22925383.0,20.473,12.657,51.97,56.232,105.426369,None,90.579948,None,2003,ZAF
20,47661514.0,24979734.0,22681781.0,20.04,11.784,53.191,57.668,103.968452,None,86.340828,None,2002,ZAF
21,47229714.0,24781908.0,22447806.0,20.392,10.871,54.565,59.47,101.990463,None,None,None,2001,ZAF
